## REX Stations Météo France horaire
### Qui parle ?
Nelly Moulin, Maitre de Conférences en Physique de l'Environnement et climatologie et Tom Lorée, Doctorant en hydrologie urbaine, tous deux à l'institut Agro Rennes-Angers.

### Le besoin
Accéder à des données météo horaire facilement. Voir où sont les stations, quelles sont les données disponibles et les télécharger

### Données source
#### Les stations sources
Les informations sur les stations sont dans un fichier à part : [téléchargement](https://meteo.data.gouv.fr/datasets/656dab84db1bdf627a40eaae), mais le fichier est un 'Geojson' qui n'est pas dans le bon format et ne s'ouvre pas (problème déjà signalé à météo france...). Le fichier donne l'emplacement des stations actuellement active. Au final ce fichier est peu utile pour la suite.

#### Les observations
Les [observations horaires de météo france](https://meteo.data.gouv.fr/datasets/6569b4473bedf2e7abad3b72), sont des csv compressés par décade et département. 
Exemple de csv: 
| NUM_POSTE 	| NOM_USUEL      	| AAAAMMJJHH 	| LAT   	| LON       	| RR1 	| FF  	| ALTI 	| T   	|
|-----------	|----------------	|------------	|-------	|-----------	|-----	|-----	|------	|-----	|
| 29075001  	| BREST-GUIPAVAS 	| 1945010106 	| 48.45 	| -4.383333 	|     	| 0.1 	| 94   	| 1.2 	|
| 29075001  	| BREST-GUIPAVAS 	| 1945010112 	| 48.45 	| -4.383333 	|     	| 0.0 	| 94   	| 5.6 	|
| 29075001  	| BREST-GUIPAVAS 	| 1945010118 	| 48.45 	| -4.383333 	|     	| 0.0 	| 94   	| 2.5 	|

Les csv sont en long = chaque variable (futur datastream) en colonne. Les coordonnées géographiques sont présentent. Certaines stations ne possèdent pas tous les datastreams. Il faut télécharger toutes les stations par une boucle qui va télécharger toutes les décades (ou du webscrapping).

In [ ]:
import requests

def dl_station(start,fin):
    base_url=f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_29_{start}-{fin}.csv.gz"
    print(base_url)
    try:
        r=requests.get(base_url)
        print(r.status_code)
        if r.status_code==200:
            open(f'/home/tloree/Documents/sofair/meteo_france_station/data/H_29_{start}-{fin}.csv.gz', 'wb').write(r.content)

    except:
        print('nein')


start=1850
fin=1859
for i in range(0,round((2010-1850)/10)+1):
    print(start,fin)
    dl_station(start,fin)

    start+=10
    fin+=10

### Création de l'instance
Création via [sofair create-instance](https://geosas.fr/sofair-dev/create-instance)

### Création de la config
Sofair permet de créer des services Sensorthing complet à partir de ce type de csv, c'est la méthode 2 fichiers (pivot json). Cette méthode permet de transformer un csv qui possède toutes les informations nécessaire en service Sensorthing complet.
Nelly va fournir les observed properties pour les variables dont elle a besoin. Le sensor sera générique.
La création des observedProperties et sensors a été faite par le code si dessous, pour le moment ce n'est pas intégré à sofair.

#### Création Observed properties et Sensors

In [ ]:
import requests
import json

url_service='https://frost.geosas.fr/station-horaire-meteo-france-test/v1.0/'

sensors=[{'name':'Station météo',
 'description':'Station météo France',
 'encodingType':'text/html',
 'metadata':'https://www.data.gouv.fr/fr/datasets/donnees-climatologiques-de-base-horaires/#/resources'}]


objet='Sensors'
for i in sensors:
    json_data = json.dumps(i)
    lien=f"{url_service}{objet}"
    print(lien)
    sensor_json=json.dumps(i)
    r=requests.post(lien,data=sensor_json)
    print(r.status_code)

observerdProperties=[
    {'name':'ff',
'description':'Wind speed',
'definition':'https://w3id.org/ozcar-theia/c_1913a530'},
    {'name':'precip',
'description':'Precipitation',
'definition':'https://w3id.org/ozcar-theia/c_1913a530'},
    {'name':'t',
'description':'Temperature',
'definition':'https://w3id.org/ozcar-theia/c_1913a530'}
]

objet='ObservedProperties'
for i in observerdProperties:
    json_data = json.dumps(i)
    lien=f"{url_service}{objet}"
    obsp_json=json.dumps(i)
    r=requests.post(lien,data=obsp_json)
    print(r.status_code)

### Création things, locations, datastreams

Création via [sofair configure-instance](https://geosas.fr/sofair-dev/configure-instance) à partir du json si dessous :

In [ ]:
{
    "pivot": {
        "things": {
            "name": "NOM_USUEL",
            "description": "NUM_POSTE",
            "properties": [
                "ALTI"
            ]
        },
        "locations": {
            "name": "NOM_USUEL",
            "description": "NOM_USUEL",
            "x": "LON",
            "y": "LAT"
        },
        "datastreams": [
            {
                "name": "FF",
                "description": "force du vent",
                "sensor.id": 1,
                "observedproperty.id": 1,
                "unitOfMeasurement": {
                    "name": "ff",
                    "symbol": "m.s-1",
                    "definition": "https://ucum.org/ucum#para-31"
                }
            },
            {
                "name": " T",
                "description": "La temperature",
                "sensor.id": 1,
                "observedproperty.id": 3,
                "unitOfMeasurement": {
                    "name": "Celcius",
                    "symbol": "°C",
                    "definition": "https://meteofrance.github.io/meteonet/english/data/ground-observations/"
                }
            },
            {
                "name": "RR1",
                "description": "Precipitation during the reporting period",
                "sensor.id": 1,
                "observedproperty.id": 2,
                "unitOfMeasurement": {
                    "name": "millimètre",
                    "symbol": "mm",
                    "definition": "https://meteofrance.github.io/meteonet/english/data/ground-observations/"
                }
            }
        ],
        "date": {
            "name": "AAAAMMJJHH",
            "format": "%Y%m%d%H"
        }
    }
}

### Ajout des observations
Ajout des données via [sofair upload-data](https://geosas.fr/sofair-dev/upload-data) à partir du json précédent et du csv des données météo précédement téléchargé. Il faut mettre le csv sur un cloud avec un accès direct poour que sofair le télécharge.

### Création du viewer
[sofair create-viewer](https://geosas.fr/sofair-dev/create-viewer) : permet de créer un [viewer](https://geosas.fr/mviewer-test/?config=https://geosas.fr/apps/sofair/station-horaire-meteo-france-test.xml#) basé sur le mviewer


### Première remarque sur le viewer en test :


Salut, c'est juste un test sur le Finistère
https://geosas.fr/mviewer-test/?config=https://geosas.fr/apps/sofair/station-horaire-meteo-france-test.xml
(il y a beaucoup de station sans données pour certaine variable à cause du formalisme du csv de météo france)
il n'y a que les 15000 premières données qui s'affichent mais la série est complete 

Ling — Aujourd’hui à 10:24
trop bien comme interface! c'est synthétique et on voit tout de suite les infos essentielles!
j'aurais une suggestion: mettre des carrés plutôt que des ronds pour la sélection des variables
(en général, rond=1seul choix , carré=plusieurs choix possibles) 

Tom — Aujourd’hui à 10:26
des carrés sur la carte ?

Ling — Aujourd’hui à 10:26
ah ok je vois...
tu penses qu'en reformatant les données en nc ou un autre format plus compact avant pourrait faciliter l'accès?

Ling — Aujourd’hui à 10:26
Image
c'est juste sur le choix d'affichage des variables ^^
en tout cas c'est super!

Tom — Aujourd’hui à 10:31
Pour le téléchargement ça ne pose pas de problème, c'est juste que la visualisation dans un navigateur d'une grande série temporelle ça peut-être un peu poussif pourles données brutes. Mais il y a des méthodes pour contourner ce genre de problème mais ça prend du temps 😉
Le sensorthing est en cours d'ajout sur Qgis également

Ling — Aujourd’hui à 10:32
et tu penses qu'en ajoutant une fenetre de temps visualisable limitée pourrait permettre d'accéder à une partie du csv à la fois?
je pense au site hub'eau par exemple qui limite l'affichage des chroniques par séries de 5ans à peu près
ou par nombre de lignes (en fonction de la résolution temporelle que tu choisis)

Tom — Aujourd’hui à 10:42
tout est possible 😉 c'est juste une question de choix

Ling — Aujourd’hui à 10:43
je pense que ça pourrait être intéressant de pouvoir choisir la résolution.
tu ne vois pas les mêmes infos sur des données horaires, moyennes journalièere, décadaire, mensuelles etc

Tom — Aujourd’hui à 10:46
pour la résolution, la question que je me pose c'est de faire le calcul à la volée ou de les  stocker

Ling — Aujourd’hui à 10:46
si tu stockes, il faut penser à tous les cas possibles en amont ^^
mmh et après, cela dépend de comment on définit la moyenne.
par exemple: moyenne journalière à partir du 01/02/2020 à 9h --> moyenne pour la journée de minuit à minuit? ou moyenne pour l'intervalle de 24h entre 9h et 9h le jour suivant
c'est encore plus vrai pour une moyenne de 10j 
à partir du jour X, tu ne peux pas te passer d'un calcul à la volée je pense
par contre, peut être possible de stocker des moyennes journalières de minuit à minuit
mensuelles du 1er au 30

et les autres moyennes, à la volée ^^
donc réponse habituelle des chercheurs: ça dépend 😝

Tom — Aujourd’hui à 10:55
😁
Mais pour une interface de visualisation simple on peut choisir quelques exemples de visualisations avant de télécharger la chronique entière

Ling — Aujourd’hui à 10:56
oui, dans ce cas, on peut se contenter de moyennes pré-calculées

Tom — Aujourd’hui à 10:57
mais en choisissant des exemples pertinents pour chaque variables
Ling — Aujourd’hui à 10:58
des exemples?

Tom — Aujourd’hui à 11:02
je ne sais pas trop, par exemple la pluie ont prend la somme de la journée. 
Pour la température en lisant la documentations pour safran les flux de données température max, min et moyenne journalier ne sont pas aggrégés à la même heure (un peu comme l'exemple que tu m'as donné) 

Ling — Aujourd’hui à 11:04
ah oui
on peut sélectionner une série de variables dérivées pour chaque variable mesurée oui
effectivement pour la T°, tu peux prendre min + heure du min (qui est une donnée importante en soit), pareil pour max, moyenne sur la journée et écart type
après il faut voir ce qui est raisonnable en nombre de variables dérivées.

par exemple pour T°, si on prend :
moyT°+minT+maxT+std T à pas de temps journalier, mensuel, annuel: on est déjà à 4 X 3 = 12 variables dérivées juste pour la T° ^^

Tom — Aujourd’hui à 11:13
Oui mais en terme de ligne ce n'est pas beaucoup.
 Autre idée : on affiche la donnée brute avec une fenêtre temporelle  de quelques années, ensuite plusieurs choix : 
télécharger la série, 
visualiser la chronique entièrement,
calcul + visualisation  de variables dérivées

Ling — Aujourd’hui à 11:14
oui bonne idée
c'est plus modulable 🙂